In [23]:
import cv2
import tkinter as tk
from tkinter import filedialog
import numpy as np
import pandas as pd
import json

In [24]:
def open_point_window(video_path):
    # Create a hidden Tkinter root window
    root = tk.Tk()
    root.withdraw()

    # Get a single frame from the video
    cap = cv2.VideoCapture(video_path)
    success, frame = cap.read()
    cap.release()

    if not success:
        print("Error: Could not read the video file.")
        return None

    # Nonlocal variable to store the selected point
    selected_point = None

    def select_point(event, x, y, flags, param):
        nonlocal selected_point  # Declare as nonlocal to modify it inside the callback
        if event == cv2.EVENT_LBUTTONDOWN:
            selected_point = (x, y)
            print(f"Point selected: {selected_point}")

    # Display the frame and set up mouse callback
    cv2.namedWindow('Select Point')
    cv2.setMouseCallback('Select Point', select_point)

    while True:
        cv2.imshow('Select Point', frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q') or selected_point is not None: 
            break

    cv2.destroyAllWindows()
    return selected_point


In [25]:
with open("/home/thomas/washu/behavior_clustering/data/Fang/processed_csvs/metadata.json", 'r') as file:
    data = json.load(file)

In [26]:
len(data["videos"].items())

11

In [ ]:
for i,k in data["videos"].items():
    video_path = k["video_path"]
    csv_path = k["csv_path"]
    point = open_point_window(video_path)
    data["videos"][i]["light_source_point"] = point

In [6]:
with open("/home/thomas/washu/behavior_clustering/data/Fang/processed_csvs/metadata.json", 'w') as file:
    file.write(json.dumps(data))

In [10]:
with open("/home/thomas/washu/behavior_clustering/data/Fang/processed_csvs/metadata.json", 'r') as file:
    data = json.load(file)

In [ ]:
for i,k in data["videos"].items():
    selected_point = k["light_source_point"]
    light_on_rgb = np.array([255, 243, 197])
    light_off_rgb = np.array([250, 160, 135])

    def is_light_on(selected_point,frame):
        x, y = selected_point
        pixel_rgb = frame[y, x]
        pixel_rgb = pixel_rgb[::-1]
        # Calculate the Euclidean distance to both the "on" and "off" reference colors
        dist_to_on = np.linalg.norm(pixel_rgb - light_on_rgb)
        dist_to_off = np.linalg.norm(pixel_rgb - light_off_rgb)

        # Return True if the light is closer to "on" color, otherwise False
        if dist_to_on < dist_to_off:
            return True
        else:
            return False  
        

    # Create dataframe
    data = []

    cap = cv2.VideoCapture(video_path)
    success, frame = cap.read()
    frame_count = 1
    while success:
        print(f"Processing frame {frame_count}")
        frame_count += 1
        light_status = is_light_on(selected_point,frame)
        data.append({"Light On": light_status})
        success, frame = cap.read()
    cap.release()
    df = pd.DataFrame(data)

    # df.to_csv(video_path[:-4] + ".csv")
    break

Processing frame 1
Processing frame 2
Processing frame 3
Processing frame 4
Processing frame 5
Processing frame 6
Processing frame 7
Processing frame 8
Processing frame 9
Processing frame 10
Processing frame 11
Processing frame 12
Processing frame 13
Processing frame 14
Processing frame 15
Processing frame 16
Processing frame 17
Processing frame 18
Processing frame 19
Processing frame 20
Processing frame 21
Processing frame 22
Processing frame 23
Processing frame 24
Processing frame 25
Processing frame 26
Processing frame 27
Processing frame 28
Processing frame 29
Processing frame 30
Processing frame 31
Processing frame 32
Processing frame 33
Processing frame 34
Processing frame 35
Processing frame 36
Processing frame 37
Processing frame 38
Processing frame 39
Processing frame 40
Processing frame 41
Processing frame 42
Processing frame 43
Processing frame 44
Processing frame 45
Processing frame 46
Processing frame 47
Processing frame 48
Processing frame 49
Processing frame 50
Processin

In [16]:
df = pd.read_csv(k["csv_path"], index_col=0)

In [22]:
k

{'csv_path': '/home/thomas/washu/behavior_clustering/data/Fang/processed_csvs/Spine_Trial/Pde1c(+) SDGC #21 (1)/Pde1c(+) SDGC #21 (1)_Pose_Data.csv',
 'video_path': '/home/thomas/washu/behavior_clustering/data/Fang/unprocessed_csvs/Pde1c(+) SDGC #21 (1).avi',
 'trial': 'Pde1c(+) SDGC #21 (1)',
 'light_source_point': [193, 118]}

In [18]:
df["Light On"] = data

ValueError: Length of values (28320) does not match length of index (22411)

In [12]:
on = df[df["Light On"] == True]

In [13]:
progressive = 0
for i, row in on.iterrows():
    if progressive == 0:
        progressive = i
        print(f"Flash Starts {(i/30)/60}")

    if i - progressive < 10:
        progressive = i
    else:
        print(f"Flash Starts {(i/30)/60}")
        progressive = i


Flash Starts 1.385
Flash Starts 1.3977777777777776
Flash Starts 3.386111111111111
Flash Starts 5.383333333333334
Flash Starts 7.386666666666667
Flash Starts 9.385
